In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
city_df = pd.read_csv("WeatherPy_challenge.csv")
city_df.head()

,City_ID,City,Country,Max Temp,Lat,Lng,Weather,Hotel Name
0,13,Busselton,AU,86.00,-33.65,115.33,light rain,Observatory Guest House
1,19,Sinnamary,GF,78.91,5.38,-52.95,light rain,ALETHEIA Appartement Sinnamary
2,20,Butaritari,KI,84.76,3.07,172.79,light rain,Isles Sunset Lodge
3,62,Taguatinga,BR,70.61,-12.39,-46.57,light rain,NaN
4,83,Tutoia,BR,72.00,-2.76,-42.27,light rain,Pousada Guará's


In [5]:
# Create dataframes for four cities (Simplicio Mendes, Pau Dos Ferros, Trairi, Tutoia)
#Simplicio Mendes
Simplicio_Mendes = city_df[city_df['City'].str.contains("Simplicio Mendes")]
Simplicio_Mendes

,City_ID,City,Country,Max Temp,Lat,Lng,Weather,Hotel Name
22,243,Simplicio Mendes,BR,70.2,-7.85,-41.91,light rain,Fidalgo


In [8]:
# Pau Dos Ferros
Pau_Dos_Ferros = city_df[city_df['City'].str.contains("Pau Dos Ferros")]
Pau_Dos_Ferros

,City_ID,City,Country,Max Temp,Lat,Lng,Weather,Hotel Name
61,523,Pau Dos Ferros,BR,73.74,-6.11,-38.21,light rain,Hertz Center Hotel


In [9]:
# Trairi
Trairi = city_df[city_df['City'].str.contains("Trairi")]
Trairi

,City_ID,City,Country,Max Temp,Lat,Lng,Weather,Hotel Name
47,426,Trairi,BR,73.31,-3.28,-39.27,light rain,Pousada Field Lilies - Trairi


In [10]:
# Tutoia
Tutoia = city_df[city_df['City'].str.contains("Tutoia")]
Tutoia

,City_ID,City,Country,Max Temp,Lat,Lng,Weather,Hotel Name
4,83,Tutoia,BR,72.0,-2.76,-42.27,light rain,Pousada Guará's


In [11]:
# Merge the four data frames
frames = [Simplicio_Mendes, Pau_Dos_Ferros, Trairi, Tutoia]
result = pd.concat(frames)
result

,City_ID,City,Country,Max Temp,Lat,Lng,Weather,Hotel Name
22,243,Simplicio Mendes,BR,70.20,-7.85,-41.91,light rain,Fidalgo
61,523,Pau Dos Ferros,BR,73.74,-6.11,-38.21,light rain,Hertz Center Hotel
47,426,Trairi,BR,73.31,-3.28,-39.27,light rain,Pousada Field Lilies - Trairi
4,83,Tutoia,BR,72.00,-2.76,-42.27,light rain,Pousada Guará's


In [12]:
four_lat_lng = result.loc[:, ['City', 'Lat', 'Lng']]
four_lat_lng

,City,Lat,Lng
22,Simplicio Mendes,-7.85,-41.91
61,Pau Dos Ferros,-6.11,-38.21
47,Trairi,-3.28,-39.27
4,Tutoia,-2.76,-42.27


In [13]:
# import gmaps
# import gmaps.datasets
# gmaps.configure(api_key='AIza...')

# # Latitude-longitude pairs
# geneva = (46.2, 6.1)
# montreux = (46.4, 6.9)
# zurich = (47.4, 8.5)

# fig = gmaps.figure()
# geneva2zurich = gmaps.directions_layer(geneva, zurich)
# fig.add_layer(geneva2zurich)
# fig

In [16]:
import gmaps
import gmaps.datasets
gmaps.configure(api_key= g_key)
SimplicioMendes = (-7.85, -41.91)
PauDosFerros = (-6.11, -38.21)
Trairi = (-3.28, -39.27)
Tutoia = (-2.76, -42.27)
fig = gmaps.figure()
SimplicioMendes2PauDosFerros = gmaps.directions_layer(SimplicioMendes, PauDosFerros)
fig.add_layer(SimplicioMendes2PauDosFerros)
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
# fig = gmaps.figure()
# geneva2zurich_via_montreux = gmaps.directions_layer(
#         geneva, zurich, waypoints=[montreux],
#         travel_mode='BICYCLING')
# fig.add_layer(geneva2zurich_via_montreux)
# fig

In [19]:
fig = gmaps.figure()
SimplicioMendes2PauDosFerrosDriving = gmaps.directions_layer(SimplicioMendes, PauDosFerros, waypoints=[Trairi, Tutoia],
                                                            travel_mode='DRIVING')
fig.add_layer(SimplicioMendes2PauDosFerrosDriving)
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
# Set the parameters to search for a hotels near locations
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}
# Use base URL to search for hotels
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [23]:
# Iterate through the DataFrame.
for index, row in result.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        result.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

In [24]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [25]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in result.iterrows()]

In [27]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = result[["Lat", "Lng"]]
max_temp = result["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))